In [1]:
import pandas as pd
import numpy as np
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import os
import re
import pickle

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


## Load pre-trained translation model from fairseq.
You can always use another middle language or other models.

In [4]:
# Load translation model
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /nethome/jchen896/.cache/torch/hub/pytorch_fairseq_master


loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-ru.single_model.tar.gz from cache at /nethome/jchen896/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949d46ae3d0a06c1be3667929101011cdc0.8699fa0753ba14331352cc26ceb7aa46fba856bad52dd40fc97d3834418bf6f1
| [en] dictionary: 31640 types
| [ru] dictionary: 31232 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/nethome/jchen896/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949d46ae3d0a06c1be3667929101011cdc0.8699fa0753ba14331352cc26ceb7aa46fba856bad52dd40fc97d3834418bf6f1/bpecodes', bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/nethome/jchen896/.cache/torch/pytorch_fairseq/57b24ff1a88d5f14807352c9d8e17949

Using cache found in /nethome/jchen896/.cache/torch/hub/pytorch_fairseq_master


loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.ru-en.single_model.tar.gz from cache at /nethome/jchen896/.cache/torch/pytorch_fairseq/f857729c1dcbbade3bbe70f7f58ede13e939a7b8278eb9015602e73be09e5cb4.ecb440f9c8cec1f28ee5023cdeab76f67eede4fc1ac0f2c70eeb2e65a25ad783
| [ru] dictionary: 31232 types
| [en] dictionary: 31640 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/nethome/jchen896/.cache/torch/pytorch_fairseq/f857729c1dcbbade3bbe70f7f58ede13e939a7b8278eb9015602e73be09e5cb4.ecb440f9c8cec1f28ee5023cdeab76f67eede4fc1ac0f2c70eeb2e65a25ad783/bpecodes', bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/nethome/jchen896/.cache/torch/pytorch_fairseq/f857729c1dcbbade3bbe70f7f58ede13

In [5]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

Using cache found in /nethome/jchen896/.cache/torch/hub/pytorch_fairseq_master


loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /nethome/jchen896/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
| [en] dictionary: 42024 types
| [de] dictionary: 42024 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/nethome/jchen896/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3/bpecodes', bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/nethome/jchen896/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320e

Using cache found in /nethome/jchen896/.cache/torch/hub/pytorch_fairseq_master


loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.de-en.joined-dict.single_model.tar.gz from cache at /nethome/jchen896/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9
| [de] dictionary: 42024 types
| [en] dictionary: 42024 types
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_wmt_en_de_big', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/nethome/jchen896/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c50d9589fd2f3cc27322e390b1ef4cf3fdcf625c0d2fd7.bf6e22453272c2cba218a5ccecd045f73e926c34c1d66c47c9b31233343820a9/bpecodes', bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/nethome/jchen896/.cache/torch/pytorch_fairseq/f42bb1b72d293668a5c5

In [6]:
en2ru.cuda()
ru2en.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (embed_tokens): Embedding(31232, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=8192, bias=True)
            (fc2): Linear(in_features=8192, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=T

In [7]:
en2de.cuda()
de2en.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=8192, bias=True)
            (fc2): Linear(in_features=8192, out_features=1024, bias=True)
            (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=T

## Load data

In [8]:
path = './'
train_df = pd.read_csv(path+'train.csv', header=None)
train_df.head()

,0,1,2
0,5,5,why doesn't an optical mouse work on a glass t...
1,6,6,what is the best off-road motorcycle trail ? l...
2,3,3,what is trans fat? how to reduce that? i heard...
3,7,7,how many planes fedex has? i heard that it is ...
4,7,7,"in the san francisco bay area, does it make se..."


In [9]:
train_labels = [v-1 for v in train_df[0]]
train_text = [v for v in train_df[2]]

In [10]:
train_text[0]

"why doesn't an optical mouse work on a glass table? or even on some surfaces? optical mice use an led and a camera to rapidly capture images of the surface beneath the mouse.  the infomation from the camera is analyzed by a dsp (digital signal processor) and used to detect imperfections in the underlying surface and determine motion. some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the dsp to accurately analyze the surface beneath the mouse.   since glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the dsp not to recognize motion properly. when the system is unable to see surface changes associated with movement, the mouse will not work properly."

In [11]:
len(train_text)

1400000

In [12]:
max(train_labels)

9

In [1]:
# split and get our unlabeled training data
def train_val_split(labels, n_labeled_per_class, n_labels, seed = 0):
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        train_labeled_idxs.extend(idxs[:n_labeled_per_class])
        train_unlabeled_idxs.extend(idxs[n_labeled_per_class : n_labeled_per_class + 10000])
        val_idxs.extend(idxs[-3000:])
    
    np.random.shuffle(train_labeled_idxs)
    np.random.shuffle(train_unlabeled_idxs)
    np.random.shuffle(val_idxs)
    return train_labeled_idxs, train_unlabeled_idxs, val_idxs

In [14]:
train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(train_labels, 500, 10)

In [15]:
len(train_unlabeled_idxs)

100000

In [16]:
idxs = train_unlabeled_idxs

In [17]:
idxs[0]

420562

## Back translation process
You can tune the temperature in the translation process to control the diversity.

In [18]:
# back translate using Russian as middle language
def translate_ru(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = ru2en.translate(en2ru.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [19]:
# back translate using German as middle language
def translate_de(start, end, file_name):
    trans_result = {}
    for id in tqdm(range(start, end)):
        trans_result[idxs[id]] = de2en.translate(en2de.translate(train_text[idxs[id]],  sampling = True, temperature = 0.9),  sampling = True, temperature = 0.9)
        if id % 500 == 0:
            with open(file_name, 'wb') as f:
                pickle.dump(trans_result, f)
    with open(file_name, 'wb') as f:
        pickle.dump(trans_result, f)

In [20]:
translate_de(0,100000, 'de_1.pkl')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed